In [1]:
#! /data/zhangxingyu/anaconda3/envs/py38/bin/python
import numpy as np
import pandas as pd
import collections
from tqdm import tqdm


In [2]:
nc_down = pd.read_csv('../data/raw/ecsmpf_round1_train_20210625/round1_train_nc_down.csv')
nc_sample_label = pd.read_csv('../data/raw/ecsmpf_round1_train_20210625/round1_train_nc_sample_label.csv')
nc_info = pd.read_csv('../data/raw/ecsmpf_round1_train_20210625/round1_train_nc_info.csv')
nc_exceptions = pd.read_csv('../data/raw/ecsmpf_round1_train_20210625/round1_train_nc_exceptions.csv')

/data/zhangxingyu/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Convert str to date
str2date = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
nc_exceptions['exception_time'] = str2date(nc_exceptions['exception_time'])

In [4]:
nc_sample_label['sample_time'] = str2date(nc_sample_label['sample_time'])

In [9]:
nc_sample_exceptions = nc_exceptions[nc_exceptions['nc_ip'].isin(nc_sample_label['nc_ip'].values)]

In [13]:
nc_sample_exceptions

,nc_ip,exception_name,exception_time,content
0,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
1,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
2,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
3,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
4,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
...,...,...,...,...
166397848,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN
166397849,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN
166397850,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN
166397851,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN


In [12]:
# nc_down_exceptions = nc_down.join(nc_exceptions.set_index('nc_ip'), on='nc_ip')
# nc_exceptions.join(nc_down)


new_tuple_list = []
column_list = ['nc_ip', 'sample_time', 'nc_down_label']
for i in tqdm(range(nc_sample_label.shape[0]), total=nc_sample_label.shape[0]):
    cur = nc_exceptions.loc[
        (nc_exceptions['nc_ip'] == nc_sample_label.loc[i]['nc_ip']) &
        (nc_exceptions['exception_time'] <= nc_sample_label.loc[i]['sample_time']) &
        (nc_sample_label.loc[i]['sample_time'] - nc_exceptions['exception_time'] <= pd.to_timedelta('60 days'))
    ]
    print(cur)


  0%|          | 1/530922 [00:18<2790:56:29, 18.92s/it]

                                         nc_ip     exception_name  \
28916485  f4724808-d7ec-4cad-9179-29414b1188b0  exception_name_58   
28916486  f4724808-d7ec-4cad-9179-29414b1188b0  exception_name_58   
28916487  f4724808-d7ec-4cad-9179-29414b1188b0  exception_name_58   
28916488  f4724808-d7ec-4cad-9179-29414b1188b0  exception_name_58   
28916489  f4724808-d7ec-4cad-9179-29414b1188b0  exception_name_58   

              exception_time content  
28916485 2020-12-19 15:23:08     NaN  
28916486 2020-12-19 15:23:08     NaN  
28916487 2020-12-19 15:23:08     NaN  
28916488 2020-12-19 15:23:08     NaN  
28916489 2020-12-19 15:23:08     NaN  


  0%|          | 1/530922 [00:27<4005:05:52, 27.16s/it]


KeyboardInterrupt: 

In [3]:
nc_ip_exceptions = nc_exceptions.groupby(['nc_ip', 'exception_name'])['exception_time'].max()

In [ ]:
nc_exceptions

Error: Session cannot generate requests

NameError: name 'nc_sample_label' is not defined

In [43]:
sample_info = nc_sample_label[nc_sample_label['nc_down_label']].groupby('nc_ip').agg(
    min_sample_time=pd.NamedAgg(column='sample_time', aggfunc='min')
)
nc_down.join(sample_info, how='left', on='nc_ip')


,nc_ip,down_start_time,is_active,min_sample_time
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True,NaN
1,e3b9909a-d9a0-4188-ad30-1d131ccb60dc,2019-11-15 09:25:24,True,2019-11-15 08:50:00
2,b539e341-4a45-4001-9aa1-f6a44f996552,2016-12-18 12:38:12,True,2016-12-17 06:30:00
3,9e0d0a75-5796-41e5-845b-558f3ef32685,2019-08-08 02:44:26,True,2019-08-07 02:50:00
4,bcb19ebd-58b6-4114-b266-2cd3df8c7867,2020-01-07 13:00:40,True,2020-01-05 13:55:00
...,...,...,...,...
201,9988b887-4fe9-48de-b207-eab2407100f8,2020-03-29 17:01:42,True,NaN
202,ab535065-0b03-4368-8065-569fdd9dc2e2,2018-04-19 06:04:47,False,2018-04-19 06:05:00
203,4ec32f4e-bd98-4865-8da2-0a06fdc6c350,2020-10-02 12:10:08,True,2020-10-02 02:05:00
204,7ec737a3-ae07-4db3-a689-90dff57010bf,2019-07-31 21:14:50,True,NaN


In [14]:
str2date = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
true_samples = nc_sample_label[nc_sample_label['nc_down_label']].groupby('nc_ip').agg(
    min_sample_time=pd.NamedAgg(column='sample_time', aggfunc='min'),
    max_sample_time=pd.NamedAgg(column='sample_time', aggfunc='max'),
    diff_sample_time=pd.NamedAgg(column='sample_time', aggfunc=lambda x: str2date(x.max()) - str2date(x.min())) 
)
true_samples.sort_values(by='diff_sample_time', ascending=False)

,min_sample_time,max_sample_time,diff_sample_time
nc_ip,,,
31bd16be-346b-4d3a-9c25-dd398bc776c4,2019-10-19 16:00:00,2019-10-21 16:00:00,2 days 00:00:00
60983cb1-4c04-4ac0-b581-bf03cb2bc326,2019-10-25 03:40:00,2019-10-27 03:40:00,2 days 00:00:00
8ef44f85-6f92-4b56-b926-9a56384dc7f6,2016-12-29 17:40:00,2016-12-31 17:40:00,2 days 00:00:00
cb90979f-90e6-4480-bd24-faf56b85959b,2016-01-31 21:15:00,2016-02-02 21:15:00,2 days 00:00:00
7befff65-7165-4aa6-81b9-78241cc66eb9,2018-03-01 18:40:00,2018-03-03 18:10:00,1 days 23:30:00
...,...,...,...
4528d582-6c35-48c0-9fe9-a5555c4aaa63,2020-01-21 10:15:00,2020-01-21 10:15:00,0 days 00:00:00
d128e721-8dba-4af6-8597-2f9629c2e000,2019-12-25 08:05:00,2019-12-25 08:05:00,0 days 00:00:00
d0610919-72e3-43b4-98b9-786a70497b75,2019-11-14 18:40:00,2019-11-14 18:40:00,0 days 00:00:00


In [44]:
nc_down.join(nc_sample_label.set_index('nc_ip'), on='nc_ip')

,nc_ip,down_start_time,is_active,sample_time,nc_down_label
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True,2020-03-05 14:10:00,False
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True,2020-03-05 14:40:00,False
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True,2020-03-05 15:15:00,False
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True,2020-03-05 14:30:00,False
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True,2020-03-05 14:15:00,False
...,...,...,...,...,...
205,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-18 02:48:45,True,2018-07-17 14:55:00,True
205,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-18 02:48:45,True,2018-07-17 14:50:00,True
205,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-18 02:48:45,True,2018-07-17 15:10:00,True
205,d33be639-de4b-4865-9fe2-d8c7ee171a2d,2018-07-18 02:48:45,True,2018-07-18 02:40:00,True


In [65]:
nc_exceptions = pd.read_csv('../data/ecsmpf_round1_train_20210625/round1_train_nc_exceptions.csv')
nc_exceptions.drop_duplicates()
# nc_info = pd.read_csv('../data/ecsmpf_round1_train_20210625/round1_train_nc_info.csv')

# nc_info

/home/zhangxingyu/.conda/envs/python38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,nc_ip,exception_name,exception_time,content
0,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
1,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
2,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
3,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
4,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
...,...,...,...,...
166397848,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN
166397849,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN
166397850,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN
166397851,eedc57fc-6756-46aa-aefa-ad89daefc127,exception_name_58,2019-10-07 07:54:35,NaN


,nc_ip,exception_name,exception_time,content
0,ab0b5660-b14b-4382-bab1-135f8f64ad64,exception_name_58,2016-03-01 00:40:53,NaN
5,d7fd7009-3d76-4354-acee-46e361b5c7f5,exception_name_58,2020-08-24 04:43:48,NaN
10,49bae709-5cda-4054-bb8d-7d2afc01e533,exception_name_32,2018-09-25 14:01:45,"OrderedDict([('bank', '3'), ('bank_group', '3'..."
11,49bae709-5cda-4054-bb8d-7d2afc01e533,exception_name_16,2018-09-29 13:06:45.016000,NaN
12,49bae709-5cda-4054-bb8d-7d2afc01e533,exception_name_32,2018-09-29 13:06:45,"OrderedDict([('bank', '3'), ('bank_group', '3'..."
...,...,...,...,...
166397844,9a568e8a-deff-42bb-b5bf-0bba7f5e6611,exception_name_47,2016-09-05 15:18:45,NaN
166397845,9a568e8a-deff-42bb-b5bf-0bba7f5e6611,exception_name_32,2016-09-05 15:18:45,"OrderedDict([('bank', '2'), ('bank_group', '1'..."
166397846,9a568e8a-deff-42bb-b5bf-0bba7f5e6611,exception_name_33,2016-09-05 15:18:45,NaN
166397847,9a568e8a-deff-42bb-b5bf-0bba7f5e6611,exception_name_57,2016-09-05 15:18:45,NaN
